In [1]:
import sys
import os

# Add the project root directory to the sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
import torch
import yaml
from models.contrastive_vae import ContrastiveVAE
from data.data_loader import load_data_from_h5, get_data_loaders


# Load the configuration file
config_path = 'D:/Ziwei/Github/LVAE/config/config.yaml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)



In [3]:
device = config['device']
experiment_dir = config['experiment_dir']
data_path = config['data']['path']
loader_params = config['loader']
model_params = config['model']['kwargs']
optimizer_params = config['optimizer']['kwargs']
criterion_params = config['criterion']
training_params = config['training']

In [4]:
data_path = config['data']['path']
train_point_clouds, test_point_clouds, train_labels, test_labels = load_data_from_h5(data_path)
batch_size = config['loader']['batch_size']

train_loader, val_loader = get_data_loaders(train_point_clouds, train_labels, test_point_clouds, test_labels, batch_size)

In [5]:
input_dim = 1024 * 3  # Assuming 1024 points with 3 coordinates each
latent_dim = config['model']['kwargs']['latent_dim']
projection_dim = config['model']['kwargs']['projection_dim']
k = config['model']['kwargs']['k']
emb_dims = config['model']['kwargs']['emb_dims']

model = ContrastiveVAE(input_dim=input_dim, latent_dim=latent_dim, projection_dim=projection_dim, k=k, emb_dims=emb_dims)
model = model.to(torch.device(config['device']))

In [6]:
from pytorch_metric_learning import losses
# Initialize optimizer
optimizer_class = getattr(torch.optim, config['optimizer']['name'])
optimizer = optimizer_class(model.parameters(), **config['optimizer']['kwargs'])

# Initialize criterion
criterion = losses.NTXentLoss(temperature=config['criterion']['temperature'])
# Test a forward pass
data_iter = iter(train_loader)
data = next(data_iter)
augmented_batch, labels = data['points'], data['label']
augmented_batch = torch.tensor(augmented_batch, dtype=torch.float32).to(torch.device(config['device']))
# Ensure augmented_batch and labels are tensors
augmented_batch = augmented_batch.clone().detach().to(torch.device(config['device']))
labels = labels.clone().detach().to(torch.device(config['device']))

C:\Users\ljd567\AppData\Local\Temp\ipykernel_15264\2170236123.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  augmented_batch = torch.tensor(augmented_batch, dtype=torch.float32).to(torch.device(config['device']))


In [6]:
from training.train import ShapeTrainer

In [7]:
trainer = ShapeTrainer(config)
trainer.run()

  0%|          | 0/217 [00:23<?, ?it/s]


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)